In [2]:
import pyodbc
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
import datetime
from datetime import date, timedelta

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)

# SQLAlchemy connectable erstellen
engine = create_engine('mssql+pyodbc://', creator=lambda: cnxn)

# Tabelle prüfen
table_name = "stock_data"  # Name der Zieltabelle
start_date = "2022-07-01"  # Startdatum, ab dem Daten überprüft werden sollen
end_date = "2023-07-01"  # Enddatum, bis zu dem Daten überprüft werden sollen

# Daten aus der Tabelle in einen DataFrame laden
select_query = f"SELECT * FROM {table_name} WHERE date >= ? AND date <= ?"
data = pd.read_sql(select_query, con=engine, params=(start_date, end_date))

# added
data["date"] = pd.to_datetime(data["date"])
data["date"] = data.index
data = data[["date", "open", "high", "low", "close", "volume", "type", "company"]]
data.reset_index(drop=True, inplace=True)

# Verbindung schließen
cnxn.close()

# DataFrame anzeigen
print(data.tail())

from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data["close"], model='multiplicative', period=30)


p, d, q = 5, 1, 2
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(data["close"], order=(p,d,q))
fitted = model.fit()

print(fitted.summary())


predictions = fitted.predict()
print(predictions)

import statsmodels.api as sm
import warnings
model=sm.tsa.statespace.SARIMAX(data['close'],
                                order=(p, d, q),
                                seasonal_order=(p, d, q, 12))
model=model.fit()
print(model.summary())

predictions = model.predict(len(data), len(data)+30)
print(predictions)

     date        open        high         low       close    volume type  \
246   246  186.830002  188.050003  185.229996  185.270004  48088700  ACT   
247   247  185.889999  188.389999  185.669998  188.059998  50730800  ACT   
248   248  187.929993  189.899994  187.600006  189.250000  51216800  ACT   
249   249  189.080002  190.070007  188.940002  189.589996  46347300  ACT   
250   250  191.630005  194.479996  191.259995  193.970001  85069600  ACT   

    company  
246    AAPL  
247    AAPL  
248    AAPL  
249    AAPL  
250    AAPL  
                               SARIMAX Results                                
Dep. Variable:                  close   No. Observations:                  251
Model:                 ARIMA(5, 1, 2)   Log Likelihood                -604.830
Date:                Fri, 07 Jul 2023   AIC                           1225.659
Time:                        18:17:55   BIC                           1253.831
Sample:                             0   HQIC                    

C:\Users\HerbyHerb\.conda\envs\tf_new\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                     SARIMAX Results                                      
Dep. Variable:                              close   No. Observations:                  251
Model:             SARIMAX(5, 1, 2)x(5, 1, 2, 12)   Log Likelihood                -591.181
Date:                            Fri, 07 Jul 2023   AIC                           1212.362
Time:                                    18:18:19   BIC                           1264.446
Sample:                                         0   HQIC                          1233.353
                                            - 251                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.6296      0.320      1.964      0.049       0.001       1.258
ar.L2          0.0679      0.332   